In [2]:
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import os
import random
from datetime import datetime, timedelta
import smtplib
import numpy as np

# Load your cleaned data
df2_clean = pd.read_csv('master_newsletter_v2.csv')

# Filter for Big Ten games: Home or Away in Big Ten conference
big10_games = df2_clean[
    (df2_clean['Home_Conference'] == 'Big Ten') | (df2_clean['Away_Conference'] == 'Big Ten')
].copy()

print(f"Total Big Ten games found: {len(big10_games)}")
print(big10_games[['Home_Team', 'Away_Team', 'Home_Conference', 'Away_Conference', 'Start_Date']].head())


Total Big Ten games found: 136
         Home_Team         Away_Team Home_Conference    Away_Conference  \
4          Rutgers            Howard         Big Ten               MEAC   
17       Minnesota    North Carolina         Big Ten                ACC   
23        Illinois  Eastern Illinois         Big Ten      Big South-OVC   
26  Michigan State  Florida Atlantic         Big Ten  American Athletic   
29       Wisconsin  Western Michigan         Big Ten       Mid-American   

            Start_Date  
4   Thu, Aug. 29, 2024  
17  Thu, Aug. 29, 2024  
23  Thu, Aug. 29, 2024  
26  Fri, Aug. 30, 2024  
29  Fri, Aug. 30, 2024  


In [3]:
# Simulate today's date as Nov 30, 2024
fake_today = datetime(2024, 12, 1)
yesterday = fake_today - timedelta(days=1)
yesterday_str = yesterday.strftime('%a, %b. %d, %Y')  # e.g. 'Sat, Nov. 29, 2025'

# Filter completed games on that date for Big Ten only
games_yesterday = df2_clean[
    (df2_clean['Completed'] == 'Yes') &
    (df2_clean['Start_Date'] == yesterday_str) &
    ((df2_clean['Home_Conference'] == 'Big Ten') | (df2_clean['Away_Conference'] == 'Big Ten'))
]

# from datetime import datetime, timedelta

# # Assuming 'Start_Date' format like 'Sat, Aug. 31, 2024'
# # Define a function to get yesterday's date in this format
# def get_yesterday_date_str():
#     yesterday = datetime.now() - timedelta(days=1)
#     # Format like 'Sat, Aug. 31, 2024'
#     return yesterday.strftime("%a, %b. %d, %Y")

# yesterday_str = get_yesterday_date_str()

# games_yesterday = df2_clean[
#     (df2_clean['Completed'] == 'Yes') &
#     (df2_clean['Start_Date'] == yesterday_str)
# ].copy()

# print(f"Games completed on {yesterday_str}: {len(games_yesterday)}")



In [4]:
def find_biggest_upset(df):
    # Only completed games with at least one team ranked (or unranked assigned 26)
    df_filtered = df[df['Completed'] == 'Yes'].copy()

    # Assign rank for unranked teams as 26 (assuming top 25 ranked)
    df_filtered['Home_Rank_Fill'] = df_filtered['Home_Rank'].fillna(26).astype(int)
    df_filtered['Away_Rank_Fill'] = df_filtered['Away_Rank'].fillna(26).astype(int)

    if df_filtered.empty:
        return None

    # Identify winner and loser and their ranks
    def get_winner_loser(row):
        if int(row['Home_Pts']) > int(row['Away_Pts']):
            return row['Home_Team'], row['Home_Rank_Fill'], row['Away_Team'], row['Away_Rank_Fill']
        else:
            return row['Away_Team'], row['Away_Rank_Fill'], row['Home_Team'], row['Home_Rank_Fill']

    df_filtered[['Winner', 'Winner_Rank', 'Loser', 'Loser_Rank']] = df_filtered.apply(
        lambda r: pd.Series(get_winner_loser(r)), axis=1)

    # Upset if winner rank worse (numerically higher) than loser rank
    df_filtered['Rank_Diff'] = df_filtered['Winner_Rank'] - df_filtered['Loser_Rank']
    upset_games = df_filtered[df_filtered['Rank_Diff'] > 0]

    if upset_games.empty:
        return None

    biggest_upset = upset_games.loc[upset_games['Rank_Diff'].idxmax()]
    return biggest_upset


In [41]:
Team_Colors = {
    "Air Force": "#003087",
    "Akron": "#041E42",
    "Alabama": "#9E1B32",
    "Appalachian State": "#222222",
    "Arizona": "#CC0033",
    "Arizona State": "#942139",
    "Arkansas": "#9D2235",
    "Arkansas State": "#CC092F",
    "Army": "#9E7E38",
    "Auburn": "#0C2340",
    "Ball State": "#BA0C2F",
    "Baylor": "#154734",
    "Boise State": "#0033A0",
    "Boston College": "#8A100B",
    "Bowling Green": "#402419",
    "Buffalo": "#005BBB",
    "BYU": "#002E5D",
    "California": "#003262",
    "Central Michigan": "#6A0032",
    "Charlotte": "#046A38",
    "Cincinnati": "#D50A0A",
    "Clemson": "#F56600",
    "Coastal Carolina": "#008E97",
    "Colorado": "#CFB87C",
    "Colorado State": "#215732",
    "Connecticut": "#003069",
    "Duke": "#00539B",
    "East Carolina": "#4B1869",
    "Eastern Michigan": "#006633",
    "FIU": "#081E3F",
    "Florida": "#FA4616",
    "Florida Atlantic": "#003366",
    "Florida State": "#782F40",
    "Fresno State": "#C41230",
    "Georgia": "#BA0C2F",
    "Georgia Southern": "#041E42",
    "Georgia State": "#0039A6",
    "Georgia Tech": "#B3A369",
    "Hawaii": "#024731",
    "Houston": "#C8102E",
    "Illinois": "#E84A27",
    "Indiana": "#990000",
    "Iowa": "#FFCD00",
    "Iowa State": "#C8102E",
    "Jacksonville State": "#D11F3E",
    "James Madison": "#450084",
    "Kansas": "#0051BA",
    "Kansas State": "#512888",
    "Kent State": "#003776",
    "Kentucky": "#0033A0",
    "Liberty": "#A41E34",
    "Louisiana": "#CE181E",
    "Louisiana-Monroe": "#882D17",
    "Louisiana Tech": "#003087",
    "Louisville": "#AD0000",
    "LSU": "#461D7C",
    "Marshall": "#00B140",
    "Maryland": "#E03A3E",
    "Memphis": "#0046AD",
    "Miami (FL)": "#F47321",
    "Miami (OH)": "#B31B1B",
    "Michigan": "#00274C",
    "Michigan State": "#18453B",
    "Middle Tennessee": "#0066CC",
    "Minnesota": "#7A0019",
    "Mississippi State": "#660000",
    "Missouri": "#F1B82D",
    "Navy": "#00205B",
    "NC State": "#CC0000",
    "Nebraska": "#E41C38",
    "Nevada": "#003366",
    "New Mexico": "#D50032",
    "New Mexico State": "#861F41",
    "North Carolina": "#4B9CD3",
    "North Texas": "#00853E",
    "Northern Illinois": "#A6192E",
    "Northwestern": "#4E2A84",
    "Notre Dame": "#C99700",
    "Ohio": "#00703C",
    "Ohio State": "#BB0000",
    "Oklahoma": "#841617",
    "Oklahoma State": "#FF7300",
    "Old Dominion": "#003057",
    "Ole Miss": "#006BA6",
    "Oregon": "#154733",
    "Oregon State": "#DC4405",
    "Penn State": "#041E42",
    "Pittsburgh": "#003594",
    "Purdue": "#CEB888",
    "Rice": "#00205B",
    "Rutgers": "#CC0033",
    "San Diego State": "#A6192E",
    "San Jose State": "#0055A2",
    "SMU": "#C0043F",
    "South Alabama": "#00205B",
    "South Carolina": "#73000A",
    "South Florida": "#006747",
    "Southern Miss": "#FFB612",
    "Stanford": "#8C1515",
    "Syracuse": "#F76900",
    "TCU": "#4D1979",
    "Temple": "#9D2235",
    "Tennessee": "#FF8200",
    "Texas": "#BF5700",
    "Texas A&M": "#500000",
    "Texas State": "#501214",
    "Texas Tech": "#CC0000",
    "Toledo": "#15397F",
    "Troy": "#7C878E",
    "Tulane": "#006747",
    "Tulsa": "#0066CC",
    "UAB": "#007A33",
    "UCF": "#BBA14F",
    "UCLA": "#2774AE",
    "UConn": "#003069",
    "UMass": "#881C1C",
    "UNLV": "#D8262C",
    "USC": "#990000",
    "UTEP": "#041E42",
    "UTSA": "#0C2340",
    "Utah": "#CC0000",
    "Utah State": "#0F2439",
    "Vanderbilt": "#866D4B",
    "Virginia": "#232D4B",
    "Virginia Tech": "#861F41",
    "Wake Forest": "#9E7E38",
    "Washington": "#4B2E83",
    "Washington State": "#981E32",
    "West Virginia": "#002855",
    "Western Kentucky": "#D40A2F",
    "Western Michigan": "#8B5B29",
    "Wisconsin": "#C5050C",
    "Wyoming": "#FFC425",
    "Abilene Christian": "#4E2A84",
    "Alabama A&M": "#88131A",
    "Alabama State": "#000000",
    "Albany": "#461D7C",
    "Alcorn State": "#582C83",
    "American International": "#FDB913",
    "Arkansas-Pine Bluff": "#231F20",
    "Austin Peay": "#CC092F",
    "Bethune-Cookman": "#5E2129",
    "Big South": "#1B365D",
    "Brown": "#4E3629",
    "Bryant": "#A89968",
    "Bucknell": "#FF5F00",
    "Butler": "#003087",
    "Cal Poly": "#0B6B3A",
    "Campbell": "#F47A20",
    "Central Arkansas": "#4F2C85",
    "Central Connecticut": "#0033A0",
    "Charleston Southern": "#002A5C",
    "Chattanooga": "#0C2340",
    "Clark Atlanta": "#B31B1B",
    "Colgate": "#862633",
    "College of Charleston": "#6C1D45",
    "Columbia": "#0088CE",
    "Cornell": "#B31B1B",
    "Dartmouth": "#00693E",
    "Davidson": "#D52B1E",
    "Dayton": "#CF0A2C",
    "Delaware": "#00539B",
    "Delaware State": "#E51B24",
    "Drake": "#0061AA",
    "Duquesne": "#041E42",
    "East Tennessee State": "#041E42",
    "Eastern Illinois": "#0033A0",
    "Eastern Kentucky": "#800000",
    "Eastern Washington": "#E30B17",
    "Elon": "#7A0019",
    "Florida A&M": "#00563F",
    "Fordham": "#890024",
    "Furman": "#582C83",
    "Gardner-Webb": "#BE1E2D",
    "Georgetown": "#818181",
    "Grambling State": "#FFD100",
    "Harvard": "#A51C30",
    "Holy Cross": "#512698",
    "Howard": "#00205B",
    "Idaho": "#B3995D",
    "Idaho State": "#E87722",
    "Illinois State": "#D31145",
    "Incarnate Word": "#E4002B",
    "Indiana State": "#0072CE",
    "Jackson State": "#002147",
    "Jacksonville State": "#D11F3E",
    "James Madison": "#450084",
    "Kennesaw State": "#FDB913",
    "Lafayette": "#781214",
    "Lamar": "#A6192E",
    "Lehigh": "#6B4C27",
    "Liberty": "#A41E34",
    "Long Island": "#7AC9E3",
    "Maine": "#003052",
    "Marist": "#C8102E",
    "Marshall": "#00B140",
    "McNeese State": "#00205B",
    "Mercer": "#FF8200",
    "Merrimack": "#142952",
    "Mississippi Valley State": "#007749",
    "Missouri State": "#4E2A84",
    "Monmouth": "#013E7C",
    "Montana": "#660000",
    "Montana State": "#003366",
    "Morehead State": "#0071BC",
    "Morgan State": "#0077C8",
    "Murray State": "#0C2340",
    "New Hampshire": "#003366",
    "Nicholls State": "#C8102E",
    "Norfolk State": "#18453B",
    "North Alabama": "#4F2C85",
    "North Carolina A&T": "#0033A0",
    "North Carolina Central": "#990000",
    "North Dakota": "#009A44",
    "North Dakota State": "#115740",
    "Northern Arizona": "#003366",
    "Northern Colorado": "#002F6C",
    "Northern Iowa": "#4F2C85",
    "Northwestern State": "#4F2C85",
    "Penn": "#990000",
    "Portland State": "#154734",
    "Prairie View A&M": "#5C068C",
    "Presbyterian": "#00205B",
    "Princeton": "#FF8F00",
    "Rhode Island": "#0098D8",
    "Richmond": "#BA0C2F",
    "Robert Morris": "#041E42",
    "Sacramento State": "#043927",
    "Sacred Heart": "#C8102E",
    "Sam Houston State": "#FF8200",
    "Samford": "#002F6C",
    "San Diego": "#003057",
    "Savannah State": "#F66733",
    "Southeast Missouri State": "#D50032",
    "Southeastern Louisiana": "#006A4D",
    "Southern": "#003087",
    "Southern Illinois": "#542E91",
    "South Carolina State": "#4F2C85",
    "South Dakota": "#A6192E",
    "South Dakota State": "#0033A0",
    "Southern Utah": "#B30C2B",
    "Stephen F. Austin": "#4F2C85",
    "Stetson": "#00563F",
    "Stonehill": "#A2AAAD",
    "Stony Brook": "#C60C30",
    "Tarleton State": "#582C83",
    "Tennessee State": "#00205B",
    "Tennessee Tech": "#7C878E",
    "Texas Southern": "#660000",
    "The Citadel": "#A7C6ED",
    "Towson": "#FFB612",
    "Troy": "#7C878E",
    "UC Davis": "#002855",
    "Utah Tech": "#D22630",
    "Valparaiso": "#866D4B",
    "Villanova": "#003366",
    "VMI": "#A6192E",
    "Wagner": "#006B54",
    "Wake Forest": "#9E7E38",
    "Weber State": "#4F2C85",
    "Western Carolina": "#633194",
    "Western Illinois": "#6F263D",
    "William & Mary": "#115740",
    "Wofford": "#A89968",
    "Yale": "#0F4D92"
}

In [40]:
# Load your CSV
df_lookup = pd.read_csv('download.csv')

# Build mapping: School name → ID (as string, to match filenames like '213.png')
team_to_code = dict(zip(df_lookup['School'], df_lookup['Id'].astype(str)))

LOGO_BASE = 'logos'

def get_logo_filename(team_name):
    code = team_to_code.get(team_name, None)
    if code is None:
        return ""
    return f"{LOGO_BASE}/{code}.png"





def build_email_header(game_df):
    game_df = game_df.copy()
    date_range = game_df['Start_Date'].unique()
    if len(date_range) == 1:
        date_str = f"Games Played: {date_range[0]}"
    else:
        date_str = f"Games Played: {date_range[0]}–{date_range[-1]}"
    n_games = len(game_df)

    # Most Exciting
    idx_max = game_df['Excitement'].astype(float).idxmax()
    row_max = game_df.loc[idx_max]
    top_game = f"{row_max['Away_Team']} @ {row_max['Home_Team']}"
    top_score = row_max['Excitement']

    # Biggest Upset
    biggest_upset = find_biggest_upset(game_df)
    if biggest_upset is not None:
        upset_text = (
            f"{biggest_upset['Away_Team']} @ {biggest_upset['Home_Team']} "
            f"({biggest_upset['Away_Pts']}-{biggest_upset['Home_Pts']})"
        )
    else:
        upset_text = "No upsets recorded."

    # Biggest Blowout (largest margin)
    game_df['Margin'] = abs(game_df['Home_Pts'].astype(int) - game_df['Away_Pts'].astype(int))
    idx_blowout = game_df['Margin'].idxmax()
    row_blowout = game_df.loc[idx_blowout]
    if int(row_blowout['Home_Pts']) >= int(row_blowout['Away_Pts']):
        blowout_game = f"{row_blowout['Home_Team']} {row_blowout['Home_Pts']}, {row_blowout['Away_Team']} {row_blowout['Away_Pts']}"
    else:
        blowout_game = f"{row_blowout['Away_Team']} {row_blowout['Away_Pts']}, {row_blowout['Home_Team']} {row_blowout['Home_Pts']}"

    header = f"""
    <div style="padding:16px 0 20px 0; text-align:center; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #1B1B1B;">
      <div style="font-size:48px; font-weight:900; letter-spacing:-2px; margin-bottom:10px; color:#004B87; text-shadow: 1px 1px 2px rgba(0,0,0,0.15);">
        Big Ten Football Daily Recap
      </div>
      <div style="font-size:20px; font-weight:600; color:#666666; margin-bottom:16px; font-style: italic;">
        {date_str} &mdash; <span style="color:#004B87;">{n_games} Games</span>
      </div>
      <div style="font-size:17px; color:#222222; margin-bottom:10px;">
        <b style="color:#E03A3E;">Most Exciting:</b> <span style="font-weight:450;">{top_game}</span> <span style="color:#288F5B; font-weight:600;">(Excitement Score: {top_score})</span>
      </div>
      <div style="font-size:17px; color:#222222; margin-bottom:10px;">
        <b style="color:#E03A3E;">Biggest Upset:</b> <span style="font-weight:450;">{upset_text}</span>
      </div>
      <div style="font-size:17px; color:#222222;">
        <b style="color:#E03A3E;">Biggest Blowout:</b> <span style="font-weight:450;">{blowout_game}</span>
      </div>
    </div>
    """
    return header








def build_email_footer(df_games, all_games):
    import pandas as pd
    import numpy as np

    # Identify the conference(s) present in current batch
    confs = df_games['Conference_Game'].dropna().unique()
    # Filter all_games to only these conferences
    filtered_games = all_games[all_games['Conference_Game'].isin(confs)]

    # Find next date after current batch
    current_max = max(pd.to_datetime(df_games['Start_Date']))
    all_dates = sorted(set(pd.to_datetime(filtered_games['Start_Date'])))
    next_dates = [d for d in all_dates if d > current_max]

    # Helper to get win percentage
    def win_pct(record):
        try:
            wins, losses = record.split(' ')[0].split('-')
            wins = int(wins)
            losses = int(losses)
            return wins / (wins + losses) if (wins + losses) > 0 else 0
        except:
            return 0

    # Compose best game text with rank
    def team_with_rank(team_name, rank):
        if pd.notnull(rank):
            # Rank in smaller font and superscript aligned, team name normal font size
            return (
                f'<span style="font-size:14px; vertical-align: super; margin-right: 3px;">{int(rank)}</span>'
                f'<span style="font-size:18px;">{team_name}</span>'
            )
        else:
            return f'<span style="font-size:18px;">{team_name}</span>'

    if next_dates:
        next_day = next_dates[0].strftime("%a, %b %d, %Y")
        # Add period to month abbreviation if needed
        months_with_period = {"Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"}
        for m in months_with_period:
            if f" {m} " in next_day:
                next_day = next_day.replace(f" {m} ", f" {m}. ")
                break

        # Filter games on the next date only within these conferences
        next_games = filtered_games[pd.to_datetime(filtered_games['Start_Date']) == next_dates[0]]

        # 1. Two ranked teams — smallest average rank
        ranked_games = next_games[
            next_games['Home_Rank'].notnull() & next_games['Away_Rank'].notnull()
        ].copy()
        if not ranked_games.empty:
            ranked_games['Avg_Rank'] = (ranked_games['Home_Rank'] + ranked_games['Away_Rank']) / 2
            candidates = ranked_games.sort_values('Avg_Rank')
        else:
            # 2. One ranked team
            one_rank_games = next_games[
                (next_games['Home_Rank'].notnull() & next_games['Away_Rank'].isnull()) |
                (next_games['Home_Rank'].isnull() & next_games['Away_Rank'].notnull())
            ].copy()
            if not one_rank_games.empty:
                # Get best individual rank in each game
                def best_ind_rank(row):
                    ranks = [r for r in [row['Home_Rank'], row['Away_Rank']] if pd.notnull(r)]
                    return min(ranks) if ranks else np.nan
                one_rank_games['Best_Rank'] = one_rank_games.apply(best_ind_rank, axis=1)
                candidates = one_rank_games.sort_values('Best_Rank')
            else:
                # 3. No ranked teams - best combined win %
                next_games = next_games.copy()
                next_games['Home_Win_Pct'] = next_games['Home_Record'].apply(win_pct)
                next_games['Away_Win_Pct'] = next_games['Away_Record'].apply(win_pct)
                next_games['Combined_Win_Pct'] = next_games['Home_Win_Pct'] + next_games['Away_Win_Pct']
                candidates = next_games.sort_values('Combined_Win_Pct', ascending=False)

        # Pick one at random from the top candidates tied for best value
        top_val = candidates.iloc[0][
            'Avg_Rank' if 'Avg_Rank' in candidates.columns else
            'Best_Rank' if 'Best_Rank' in candidates.columns else
            'Combined_Win_Pct'
        ]
        top_candidates = candidates[
            candidates[
                'Avg_Rank' if 'Avg_Rank' in candidates.columns else
                'Best_Rank' if 'Best_Rank' in candidates.columns else
                'Combined_Win_Pct'
            ] == top_val
        ]
        best_game = top_candidates.sample(1).iloc[0]

        best_game_text = (
            f"{team_with_rank(best_game['Away_Team'], best_game['Away_Rank'])} "
            f"vs {team_with_rank(best_game['Home_Team'], best_game['Home_Rank'])}"
        )

        n_next = len(next_games)
        next_line = f"<b>Next Game(s):</b> {next_day} — {n_next} Scheduled"
        best_game_line = f"<b>Best Upcoming Game:</b> {best_game_text}"

    else:
        next_line = "<b>No more games scheduled.</b>"
        best_game_line = ""

    footer = f"""
    <div style="margin-top:12px; padding:12px 0 32px 0; text-align:center; color:#000000; font-size:18px;">
      {next_line}<br>
      {best_game_line}<br><br>
      <span style="color:#000080; font-size:18px;">Thank You for Reading!</span>
      <br>
      <span style="color:#888; font-size:14px; display:block; margin-top:7px;">Will Semmer</span>
    </div>
    """
    return footer











def make_game_card_email(row, home_cid, away_cid):
    home_color = Team_Colors.get(row['Home_Team'], '#222')
    away_color = Team_Colors.get(row['Away_Team'], '#222')
    home_score_color = home_color
    away_score_color = away_color

    # Only show #Rank if team is ranked; blank otherwise
    def display_rank(rank, color):
        return (
            f'<span style="font-size:14px; font-weight:400; color:{color}; vertical-align:middle; margin-right:3px;">'
            f'{int(rank)}</span> '
            if pd.notnull(rank) else ''
        )

    card = f"""
    <table width="600" cellpadding="0" cellspacing="0" border="0" 
        style="
            background: linear-gradient(to bottom, #fffdf8, #f0f0e9);
            border:2.5px solid #b6b6b6; 
            border-radius:16px; 
            margin-bottom:32px; 
            font-family:'Segoe UI',Arial,sans-serif;
            box-shadow:0 8px 32px #d8d8d8;">
      <tr>
        <!-- Away team -->
        <td width="32%" align="center" style="padding:22px 10px 18px 10px;">
          <img src="cid:{away_cid}" width="64" height="64"
            style="border-radius:50%; border:2.5px solid #c8c8c8; margin-bottom:12px; box-shadow:0 2px 8px #ddd;">
          <div style="font-size:22px; font-weight:600; margin-top:8px; margin-bottom:2px; line-height:1.1; color:{away_color};">
            {display_rank(row['Away_Rank'], away_color)}{row['Away_Team']}
          </div>
          <div style="font-size:12.5px; color:#888; margin-bottom:4px;">
              {row['Away_Record']} ({row['Away_Conf_Record']})
          </div>
          <div style="font-size:30px; font-weight:bold; color:{away_score_color}; margin-top:6px;">{row['Away_Pts']}</div>
        </td>
        <!-- Score box and details -->
        <td width="36%" align="center" style="padding:18px 0 14px 0;">
          <table cellpadding="0" cellspacing="0" border="0"
            style="
              border:1.5px solid #222;
              border-radius:14px;
              font-size:15px;
              width:260px;
              min-width:260px;
              margin:0 auto;
              background:#f6f9fc;
              overflow:hidden;">
            <tr style="background-color:#d7ebfa;">
              <th style="border:none; padding:5px 7px; border-top-left-radius:14px;"></th>
              <th style="border:none; color:#1434A4; padding:5px 7px;">1Q</th>
              <th style="border:none; color:#1434A4; padding:5px 7px;">2Q</th>
              <th style="border:none; color:#1434A4; padding:5px 7px;">3Q</th>
              <th style="border:none; color:#1434A4; padding:5px 7px;">4Q</th>
              <th style="border:none; background:#1434A4; color:#fff; padding:5px 16px; border-top-right-radius:14px;">T</th>
            </tr>
            <tr style="background-color:#fff;">
              <td align="center" style="border:none; padding:4px 7px; font-size:14px; color:{away_color};">{row['Away_Team']}</td>
              <td align="center" style="border:none;">{row['Away_1Q']}</td>
              <td align="center" style="border:none;">{row['Away_2Q']}</td>
              <td align="center" style="border:none;">{row['Away_3Q']}</td>
              <td align="center" style="border:none;">{row['Away_4Q']}</td>
              <td align="center" style="border:none; font-weight:bold; padding:4px 16px;">{row['Away_Pts']}</td>
            </tr>
            <tr style="background-color:#f5fbff;">
              <td align="center" style="border:none; padding:4px 7px; font-size:14px; color:{home_color}; border-bottom-left-radius:14px;">{row['Home_Team']}</td>
              <td align="center" style="border:none;">{row['Home_1Q']}</td>
              <td align="center" style="border:none;">{row['Home_2Q']}</td>
              <td align="center" style="border:none;">{row['Home_3Q']}</td>
              <td align="center" style="border:none;">{row['Home_4Q']}</td>
              <td align="center" style="border:none; font-weight:bold; padding:4px 16px; border-bottom-right-radius:14px;">{row['Home_Pts']}</td>
            </tr>
          </table>
          <div style="margin-top:10px; color:#444; font-size:13.5px; line-height:1.5;">
            <div style="margin:0; padding:0; font-size:13.5px;">{row['Start_Date']} &bull; {row['Start_Time']} ET</div>
            <div style="
              font-weight:bold;
              margin:0;
              padding:0;
              font-size:14px;
              text-align:center;
              display:inline-block;
              max-width:220px;
              white-space:nowrap;
              overflow:hidden;
              text-overflow:ellipsis;
            ">{row['Venue']}</div>
            <div style="margin:0; padding:0; font-size:13.5px;">
              <span style="font-weight:bold;">{row['Conference_Game']}</span> | Excitement: <b>{row['Excitement']}</b>
            </div>
          </div>
        </td>
        <!-- Home team -->
        <td width="32%" align="center" style="padding:22px 10px 18px 10px;">
          <img src="cid:{home_cid}" width="64" height="64"
            style="border-radius:50%; border:2.5px solid #c8c8c8; margin-bottom:12px; box-shadow:0 2px 8px #ddd;">
          <div style="font-size:22px; font-weight:600; margin-top:8px; margin-bottom:2px; line-height:1.1; color:{home_color};">
            {display_rank(row['Home_Rank'], home_color)}{row['Home_Team']}
          </div>
          <div style="font-size:12.5px; color:#888; margin-bottom:4px;">
              {row['Home_Record']} ({row['Home_Conf_Record']})
          </div>
          <div style="font-size:30px; font-weight:bold; color:{home_score_color}; margin-top:6px;">{row['Home_Pts']}</div>
        </td>
      </tr>
    </table>
    """
    return card


In [43]:
# Filter for yesterday's Big 10 completed games
games_yesterday = big10_games[
    (big10_games['Completed'] == 'Yes') &
    (big10_games['Start_Date'] == yesterday_str)
].copy()

# Calculate Min_Rank for sorting
def min_rank(row):
    ranks = []
    if pd.notnull(row['Home_Rank']):
        ranks.append(row['Home_Rank'])
    if pd.notnull(row['Away_Rank']):
        ranks.append(row['Away_Rank'])
    return min(ranks) if ranks else np.nan

games_yesterday['Min_Rank'] = games_yesterday.apply(min_rank, axis=1)
games_yesterday['Min_Rank_Fill'] = games_yesterday['Min_Rank'].fillna(9999)

# Sort by Min_Rank ascending, then Excitement descending
games_yesterday_sorted = games_yesterday.sort_values(
    by=['Min_Rank_Fill', 'Excitement'], ascending=[True, False]
).reset_index(drop=True)

logo_files = []
cid_lookup = {}

if games_yesterday_sorted.empty:
    print(f"No completed games on {yesterday_str}. No email will be sent.")
else:
    report_html = build_email_header(games_yesterday_sorted)

    for idx, (_, row) in enumerate(games_yesterday_sorted.iterrows(), 1):
        # conf = row['Conference_Game']
        # if conf and conf.lower() != "non-conf":
        #     game_label = f"Game #{idx} ({conf}):"
        # else:
        #     game_label = f"Game #{idx}:"

        home_team = row['Home_Team']
        away_team = row['Away_Team']

        home_logo = get_logo_filename(home_team)
        away_logo = get_logo_filename(away_team)

        for team, logo_file, label in [(away_team, away_logo, 'away'), (home_team, home_logo, 'home')]:
            if logo_file is None or not os.path.isfile(logo_file):
                continue
            if (team, label) not in cid_lookup:
                cid = f"logo_{team.replace(' ', '').replace('&','').replace('(','').replace(')','')[:12]}_{label}"
                cid_lookup[(team, label)] = cid
                logo_files.append((cid, logo_file))

        home_cid = cid_lookup.get((home_team, 'home'), '')
        away_cid = cid_lookup.get((away_team, 'away'), '')

        report_html += (
            '<table width="100%" border="0" cellspacing="0" cellpadding="0" style="margin:0 auto;">'
            '<tr><td align="center">'
            f'{make_game_card_email(row, home_cid, away_cid)}'
            '</td></tr></table>'
        )


    report_html += build_email_footer(games_yesterday_sorted, df2_clean)

    final_html = f"""
    <table width="750" align="center" cellpadding="0" cellspacing="0" border="0" 
      style="background:#e8f4f8; border:4px solid #d0d0d0; border-radius:18px; box-shadow:0 8px 40px #d8d8e8; margin: 30px auto;">
      <tr>
        <td style="padding: 40px 16px 32px 16px;">
          {report_html}
        </td>
      </tr>
    </table>
    """




def send_email_with_logos(subject, html_body, logo_files, recipients, sender,
                          smtp_server, smtp_port, smtp_user, smtp_password):
    # Create email message container
    msg = MIMEMultipart('related')
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    # Create the HTML part (multipart/alternative for compatibility)
    msg_alternative = MIMEMultipart('alternative')
    msg.attach(msg_alternative)

    # Attach the HTML content
    msg_text = MIMEText(html_body, 'html')
    msg_alternative.attach(msg_text)

    # Attach each logo as inline image with Content-ID
    for cid, file_path in logo_files:
        if os.path.isfile(file_path):
            with open(file_path, 'rb') as img:
                img_data = img.read()
                img_mime = MIMEImage(img_data)
                img_mime.add_header('Content-ID', f'<{cid}>')
                img_mime.add_header('Content-Disposition', 'inline', filename=os.path.basename(file_path))
                msg.attach(img_mime)
        else:
            print(f"Logo file not found: {file_path}")

    # Send the email using SMTP SSL
    with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
        server.login(smtp_user, smtp_password)
        server.sendmail(sender, recipients, msg.as_string())

In [46]:
if not games_yesterday.empty:
    send_email_with_logos(
        subject = f"Big 10 Football Daily Recap for {yesterday_str}",
        html_body = final_html,
        logo_files = logo_files,
        recipients = ["will.semmer@gmail.com"],
        sender = "will.semmer@gmail.com",
        smtp_server = "smtp.gmail.com",
        smtp_port = 465,
        smtp_user = "will.semmer@gmail.com",
        smtp_password = "lzdf ngyw lvqg trlv"
    )
else:
    print(f"No completed games on {yesterday_str} to send.")
